In [1]:
!pip install datasets transformers accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import datasets
import torch

import evaluate

from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [3]:
torch.manual_seed(123)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
#tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-mini-finetuned-age_news-classification")
#model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-mini-finetuned-age_news-classification", num_labels=4)


tokenizer = AutoTokenizer.from_pretrained("sileod/deberta-v3-base-tasksource-nli")
model = AutoModelForSequenceClassification.from_pretrained("sileod/deberta-v3-base-tasksource-nli")


#tokenizer = AutoTokenizer.from_pretrained("dfurman/deberta-v3-base-imdb")
#model = AutoModelForSequenceClassification.from_pretrained("dfurman/deberta-v3-base-imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [6]:
model = model.to(device)

In [7]:
#dataset = datasets.load_dataset('ajaykarthick/imdb-movie-reviews')
dataset = datasets.load_dataset("stanfordnlp/imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256, return_tensors='pt').to(device)


In [9]:
train = dataset['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
val = dataset['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    per_device_train_batch_size=8
    )

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
trainer.train()

Step,Training Loss
500,0.341900
1000,0.295900
1500,0.277200
2000,0.237100
2500,0.213700
3000,0.194000


TrainOutput(global_step=3125, training_loss=0.25855738159179686, metrics={'train_runtime': 1807.3361, 'train_samples_per_second': 13.833, 'train_steps_per_second': 1.729, 'total_flos': 3288976704000000.0, 'train_loss': 0.25855738159179686, 'epoch': 1.0})

In [14]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, device='cuda'):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(self.device)
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
labels = val['label']
clear_val = val.map(remove_columns=['text', 'label'])
clear_val.set_format('pt')

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [16]:
clear_val

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [17]:
val_loader = DataLoader(clear_val, batch_size=8)

In [18]:
predict = []
for batch in val_loader:
  output = model(input_ids=batch['input_ids'].to(device),
                 token_type_ids=batch['token_type_ids'].to(device),
                 attention_mask=batch['attention_mask'].to(device))
  predict.extend(torch.argmax(output.logits, dim=1))

In [19]:
acc = evaluate.load('accuracy')

In [20]:
print(acc.compute(predictions=predict, references=labels))

{'accuracy': 0.94304}
